<a href="https://colab.research.google.com/github/oshan35/Epileptic-Seizure-Prediction-with-EEG/blob/main/Time_Clip_Segments_Non_Sezisure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mne
!pip install numpy
!pip install matplotlib
!pip install pandas
!pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.3 MB/s eta 0:00:00


In [33]:
import logging

def create_log():
  for handler in logging.root.handlers[:]:
      logging.root.removeHandler(handler)

  logging.basicConfig(filename='preprocess-pipeline.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
create_log()

In [4]:
import re

class EEGData:
    def __init__(self, file_name, file_start_time, file_end_time, num_seizures, seizure_times=[]):
        self.file_name = file_name
        self.file_start_time = file_start_time
        self.file_end_time = file_end_time
        self.num_seizures = num_seizures
        self.seizure_times = seizure_times


    def __repr__(self):
        return (f"EEGData(File Name: {self.file_name}, "
                f"File Start Time: {self.file_start_time}, "
                f"File End Time: {self.file_end_time}, "
                f"Number of Seizures: {self.num_seizures}, "
                f"Number of Seizures: {self.seizure_times}")

In [ ]:
def parse_eeg_data(file_path):
    eeg_data_objects = []

    with open(file_path, 'r') as file:
        lines = file.readlines()

    current_file_data = {}
    for line in lines:
        if line.startswith('File Name:'):
            if current_file_data:
                # Create EEGData object from the current_file_data
                eeg_data = EEGData(**current_file_data)
                eeg_data_objects.append(eeg_data)

                current_file_data = {}
            current_file_data['file_name'] = line.split(': ')[1].strip()
        elif line.startswith('File Start Time:'):
            current_file_data['file_start_time'] = line.split(': ')[1].strip()
        elif line.startswith('File End Time:'):
            current_file_data['file_end_time'] = line.split(': ')[1].strip()
        elif line.startswith('Number of Seizures in File:'):
            current_file_data['num_seizures'] = int(line.split(': ')[1].strip())
        elif line.startswith('Seizure Start Time:'):
            current_file_data['seizure_start_time'] = line.split(': ')[1].strip()
        elif line.startswith('Seizure End Time:'):
            current_file_data['seizure_end_time'] = line.split(': ')[1].strip()
            # In case the last file also has seizure information, we need to ensure it gets added
            if 'file_name' in current_file_data:
                eeg_data = EEGData(**current_file_data)
                eeg_data_objects.append(eeg_data)
                current_file_data = {}

    return eeg_data_objects


In [40]:
def parse_eeg_data(file_path):
    eeg_data_objects = []

    with open(file_path, 'r') as file:
        lines = file.readlines()
    current_file_data = {}
    num_seizures = 0
    line_index = 0
    while line_index<len(lines):
      line = lines[line_index]
      if line.startswith('File Name:'):
          if current_file_data:
              # Create EEGData object from the current_file_data
              eeg_data = EEGData(**current_file_data)
              eeg_data_objects.append(eeg_data)

              current_file_data = {}
          current_file_data['file_name'] = line.split(': ')[1].strip()
      elif line.startswith('File Start Time:'):
          current_file_data['file_start_time'] = line.split(': ')[1].strip()
      elif line.startswith('File End Time:'):
          current_file_data['file_end_time'] = line.split(': ')[1].strip()
      elif line.startswith('Number of Seizures in File:'):
          current_file_data['num_seizures'] = int(line.split(': ')[1].strip())
          num_seizures = current_file_data['num_seizures']

          seizure_times = []

          if num_seizures > 1:

            for x in range(num_seizures):
              line_index += 1
              start = int(lines[line_index].split(": ")[1].strip().split(" ")[0].strip())
              line_index += 1
              end = int(lines[line_index].split(": ")[1].strip().split(" ")[0].strip())
              seizure = (start,end)
              seizure_times.append(seizure)

          elif num_seizures  == 1:
            line_index += 1
            start = int(lines[line_index].split(": ")[1].split(" ")[0].strip())
            line_index += 1
            end = int(lines[line_index].split(": ")[1].split(" ")[0].strip())
            seizure_times.append((start,end))

          current_file_data['seizure_times'] = seizure_times

          if 'file_name' in current_file_data:
              eeg_data = EEGData(**current_file_data)
              eeg_data_objects.append(eeg_data)
              current_file_data = {}
      line_index += 1
    return eeg_data_objects


In [41]:
patient_id = "chb12"
arr = parse_eeg_data(f'/content/drive/MyDrive/EEG-Projects/CHB-MIT/{patient_id}/{patient_id}-summary.txt')
for item in arr:
  print(item)

EEGData(File Name: chb12_06.edf, File Start Time: 22:44:34, File End Time: 23:44:40, Number of Seizures: 2, Number of Seizures: [(1665, 1726), (3415, 3447)]
EEGData(File Name: chb12_08.edf, File Start Time: 00:46:38, File End Time: 1:46:38, Number of Seizures: 4, Number of Seizures: [(1426, 1439), (1591, 1614), (1957, 1977), (2798, 2824)]
EEGData(File Name: chb12_09.edf, File Start Time: 01:46:45, File End Time: 2:46:57, Number of Seizures: 2, Number of Seizures: [(3082, 3114), (3503, 3535)]
EEGData(File Name: chb12_10.edf, File Start Time: 02:47:45, File End Time: 3:47:56, Number of Seizures: 2, Number of Seizures: [(593, 625), (811, 856)]
EEGData(File Name: chb12_11.edf, File Start Time: 03:48:28, File End Time: 4:28:59, Number of Seizures: 1, Number of Seizures: [(1085, 1122)]
EEGData(File Name: chb12_19.edf, File Start Time: 08:57:21, File End Time: 9:57:21, Number of Seizures: 0, Number of Seizures: []
EEGData(File Name: chb12_20.edf, File Start Time: 09:57:27, File End Time: 10:5

In [42]:
import glob
import mne
import gc
from tqdm import tqdm
import numpy as np

def load_data_from_file(patient_id,file_name):
  file = f"/content/drive/MyDrive/EEG-Projects/CHB-MIT/{patient_id}/{file_name}"
  data = mne.io.read_raw_edf(file)
  logging.info(f"loaded EDF file at {file_name} with patient id : {patient_id}")
  return data

In [43]:
import random

def clean_channel_data(data_file):
  data = data_file.get_data()
  logging.info(f"data shape before cleaning: {data.shape}")

  modified_data = []
  channel_num, data_num = data.shape
  if channel_num > 23:
    additional_channel = True
    for channel_index in range(len(data)):
      element = data[channel_index][random.randint(0,data_num-1)]

      if element != -1e-06:
        modified_data.append(data[channel_index])
      else:
        logging.info(f"removed channel index {channel_index}")

    if len(modified_data) > 23:
      modified_data.pop(-1)
      logging.info(f"removed ECG channel")

    new_data = np.array(modified_data)
    logging.info(f"data shape changed from {data.shape} => {new_data.shape}")
    del modified_data,data
    gc.collect()
    return new_data

  logging.info("data is clean")
  return data


# x = load_data_from_file("chb04","chb04_07.edf")
# clean_channel_data(x)

In [64]:
def create_non_sezizer_segments(patient_id,eeg_data_objects, segment_length = 7200, time_gap =900):
  count = 0
  for i in range(len(eeg_data_objects)):
    if (int(eeg_data_objects[i].num_seizures) == 0) and i-4 >= 0 and i+3 < len(eeg_data_objects):
      logging.info(f"[+] non-seizure: {eeg_data_objects[i]}")
      flag = True
      index = i-1
      for j in range(4):
        if eeg_data_objects[index].num_seizures != 0:
          flag = False
        index-=1
      index = i
      for k in range(3):
        if eeg_data_objects[index].num_seizures != 0:
          flag = False
        index+=1

      if flag:
        print(eeg_data_objects[i],eeg_data_objects[i-1])
        obj1 = clean_channel_data(load_data_from_file(patient_id,eeg_data_objects[i].file_name))

        obj2 = clean_channel_data(load_data_from_file(patient_id,eeg_data_objects[i-1].file_name))


        f1_no = eeg_data_objects[i].file_name.split('_')[1].split('.')[0]
        f2_no = eeg_data_objects[i-1].file_name.split('_')[1].split('.')[0]

        conect_data = np.concatenate((obj1, obj2), axis=1)
        #np.save(f"/content/drive/MyDrive/EEG-Projects/CHB-MIT-2hour-Segments-Non-Seizure/{patient_id}-{f1_no}-{f2_no}.npy",conect_data)
        del obj1
        del obj2
        del conect_data
        gc.collect()
    elif (int(eeg_data_objects[i].num_seizures) > 0) and (i-2 >= 0):
      logging.info(f"[+] seizure: {eeg_data_objects[i]}")
      flag = True


      obj1 = clean_channel_data(load_data_from_file(patient_id,eeg_data_objects[i].file_name))
      obj2 = clean_channel_data(load_data_from_file(patient_id,eeg_data_objects[i-1].file_name))
      obj3 = clean_channel_data(load_data_from_file(patient_id,eeg_data_objects[i-2].file_name))
      time_correction = len(obj2[0])/256 + len(obj3[0])/256
      shape_flag = False

      if obj1.shape[0] == obj2.shape[0] == obj3.shape[0]:

        shape_flag = True

      obj4 = None
      if i-3 >= 0:
        obj4 = clean_channel_data(load_data_from_file(patient_id,eeg_data_objects[i-3].file_name))
        if obj1.shape[0] != obj4.shape[0] and shape_flag == True:
          shape_flag = False

        time_correction += len(obj4[0])/256
      logging.info(f"data shapes: {obj1.shape} {obj2.shape} {obj3.shape}")
      if shape_flag == False:
        logging.info("Shapes doesn't match!")
        continue

      logging.info(f"time correction: {time_correction}")

      timeline = []
      if np.all(obj4 != None):
        timeline = np.concatenate((obj4,obj3,obj2,obj1),axis=1)
      else:
        timeline = np.concatenate((obj3,obj2,obj1),axis=1)

      logging.info(timeline.shape)

      s_count = 0
      for s in eeg_data_objects[i].seizure_times:
        seizure_data = []

        start_time = s[0] + time_correction
        end_time = s[1] + time_correction
        logging.info(f"capture margin: {start_time} - {end_time}")

        end_margin = start_time - time_gap
        start_margin = end_margin - segment_length

        start_data_index = int(start_margin * 256)
        end_data_index = int(end_margin * 256)

        for x in range(23):

          channel_data = timeline[x][start_data_index:end_data_index]
          seizure_data.append(channel_data)
        s_count+=1

      logging.info(f"number of sezuers: {s_count}")



      logging.info(f"data length: {len(seizure_data)}")
      del obj1,obj2,obj3,obj4,seizure_data
      gc.collect()
    count+=1


In [65]:
def load_two_hour_segments(patient_id):
  eeg_data_objects = parse_eeg_data(f'/content/drive/MyDrive/EEG-Projects/CHB-MIT/{patient_id}/{patient_id}-summary.txt')
  create_non_sezizer_segments(patient_id,eeg_data_objects)

  del eeg_data_objects
  gc.collect

In [72]:
gc.collect()

0

In [73]:
!rm "preprocess-pipeline.log"
create_log()

In [71]:
patient_id = "chb23"
load_two_hour_segments(patient_id)

Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb23/chb23_08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-42-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb23/chb23_07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-42-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb23/chb23_06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

<ipython-input-42-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb23/chb23_07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-42-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb23/chb23_06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-42-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)


In [74]:
for i in range(24):
  patient_id = f"chb{i+1:02d}"
  load_two_hour_segments(patient_id)

Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb01/chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-42-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb01/chb01_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-42-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb01/chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb12/chb12_40.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-42-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)
<ipython-input-42-e796011988ef>:9: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb12/chb12_39.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-42-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)
<ipython-input-42-e796011988ef>:9: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  data = mne.io.read_raw_edf(file)


KeyboardInterrupt: ignored